# Imports

In [1]:
import pandas as pd
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Data Set Prep

In [2]:
df_requests = pd.read_excel('frc_data.xlsx', sheet_name='frc data')
#df_requests_condensed = pd.read_excel('frc_data.xlsx', sheet_name='frc_condensed')
# df_people = pd.read_excel('FRC Data1.xlsx', sheet_name='Requestor')

In [3]:
df_requests['Category'].value_counts()

Process    4010
Systems    1975
Forms      1587
Rate       1235
Other       505
Name: Category, dtype: int64

In [4]:
df_requests.head()

,ID,Submission Date,Resolution Date,FRC Owner,Request,Requestor,Request Description,Status,Urgent?,Reopen?,...,Incident / PME / Project # (Where Appl.),Created,Created By,Modified,Last Modified By,Submission Month,res date,Comments/Notes,Item Type,Path
0,14632,2018-10-18,2018-10-18,"Iannacone, Kristine",Account locked in TAP,"Melaniphy,Kevin M","\n​\nTeam,\n \nMy policy is locked in TAP, can...",Closed,No,No,...,NaN,2018-10-18 16:46:51,"Iannacone,Kristine",2018-10-18 17:02:04,"Iannacone,Kristine",201810,10/18/2018 17:02,"\nGood afternoon Kevin,\nThe TAP helpdesk will...",Item,ccm/FRC/Lists/TEST
1,14631,2018-10-18,NaT,"Osswald, Ernst",Can we add Tow & Labor to auto without PPTs?,"Loquiao,Jeffrey E","\n​\nHello,\n \nCan you please confirm if we a...",Open,No,No,...,NaN,2018-10-18 16:42:54,"Iannacone,Kristine",2018-10-18 16:42:54,"Iannacone,Kristine",201810,-,NaN,Item,ccm/FRC/Lists/TEST
2,14630,2018-10-18,NaT,"Ewing, Jamie",BPP rates higher than Property rates,"Williams,Gina","\n​\nHello,\n \nCan you look into why the BPP ...",Open,No,No,...,NaN,2018-10-18 16:32:44,"Iannacone,Kristine",2018-10-18 16:32:44,"Iannacone,Kristine",201810,-,NaN,Item,ccm/FRC/Lists/TEST
3,14629,2018-10-18,NaT,"Ewing, Jamie",Problem saving files to EPC/DMF,"Albright,Kevin T.",\n​I thought I read something recently on this...,Open,No,No,...,NaN,2018-10-18 16:25:37,"Iannacone,Kristine",2018-10-18 16:25:37,"Iannacone,Kristine",201810,-,NaN,Item,ccm/FRC/Lists/TEST
4,14628,2018-10-18,NaT,"Iannacone, Kristine",Need point of contact for WC claims issues,"Sterling,Heather","\n​\nHello,\n \nI have a insured who would lik...",Open,No,No,...,NaN,2018-10-18 16:16:11,"Iannacone,Kristine",2018-10-18 16:16:11,"Iannacone,Kristine",201810,-,NaN,Item,ccm/FRC/Lists/TEST


## Condense LOB Categories

In [5]:
monoline_list = ['Equipment Breakdown', 'Inland Marine', 'International', 'Ocean Marine']
package_general_liability_list = ['Connect CNP', 'General Liability', 'Multiline']
package_property_list = ['Paramount Package', 'Property']
condensed_lob_list = []
for row in df_requests['LOB']:
    if row in monoline_list:
        condensed_lob_list.append('Monoline')
    elif row in package_general_liability_list:
        condensed_lob_list.append('Package General Liability')
    elif row in package_property_list:
        condensed_lob_list.append('Package Property')
    else:
        condensed_lob_list.append(row)
        
df_requests['Updated LOB'] = condensed_lob_list

In [6]:
df_requests['Updated LOB'].value_counts()

Package General Liability    2407
Auto                         1948
Workers Comp                 1665
Package Property             1438
Umbrella                      878
Not LOB Specific              561
Monoline                      416
Name: Updated LOB, dtype: int64

### Test Classification without RST Help

In [5]:
# df_condensed = df_requests.drop(df_requests[df_requests.Channel == 'RST Help'].index)

# monoline_list = ['Equipment Breakdown', 'Inland Marine', 'International', 'Ocean Marine']
# package_general_liability_list = ['Connect CNP', 'General Liability', 'Multiline']
# package_property_list = ['Paramount Package', 'Property']
# condensed_lob_list = []
# for row in df_condensed['LOB']:
#     if row in monoline_list:
#         condensed_lob_list.append('Monoline')
#     elif row in package_general_liability_list:
#         condensed_lob_list.append('Package General Liability')
#     elif row in package_property_list:
#         condensed_lob_list.append('Package Property')
#     else:
#         condensed_lob_list.append(row)
# df_condensed['Updated LOB'] = condensed_lob_list

## Reduce Keywords

In [ ]:
key_word_list = ['Rate Change',
'Form Review',
'Endorsement Print',
'Premium Discrepancy',
'CAT',
'Endorsement Process',
'Experience Modification',
'Billing',
'System Issue']
condensed_list = []
for row in df_requests['Key Word Roll-Up']:
    if row in key_word_list:
        condensed_list.append(row)
    else: 
        condensed_list.append('Other')
        
df_requests['key_words'] = condensed_list

## Keras Text Classification - LOB

In [27]:
train_size = int(len(df_requests) * .8)
train_posts = df_requests['Request Description'][:train_size]
train_tags = df_requests['Updated LOB'][:train_size]
test_posts = df_requests['Request Description'][train_size:]
test_tags = df_requests['Updated LOB'][train_size:]

vocab_size = 1000
tokenize = text.Tokenizer(num_words=vocab_size)
tokenize.fit_on_texts(train_posts)

x_train = tokenize.texts_to_matrix(train_posts)

encoder = LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

model = Sequential()

model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))

model.add(Dense(y_train.shape[1]))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

history = model.fit(x_train, y_train, 
                    batch_size=100, 
                    epochs=2, 
                    verbose=1, 
                    validation_split=0.1)

Train on 6705 samples, validate on 745 samples
Epoch 1/2
6705/6705 [==============================] - 9s 1ms/step - loss: 1.4939 - acc: 0.4786 - val_loss: 1.3127 - val_acc: 0.5557
Epoch 2/2
6705/6705 [==============================] - 8s 1ms/step - loss: 0.9144 - acc: 0.7028 - val_loss: 1.1748 - val_acc: 0.5799


In [30]:
predictions = model.predict(x_train)
np.argmax(predictions[0])

0

In [31]:
predicted_labels = [np.argmax(x) for x in predictions]
predicted_labels

[0,
 0,
 4,
 4,
 6,
 6,
 6,
 0,
 3,
 0,
 5,
 3,
 2,
 6,
 6,
 2,
 4,
 0,
 0,
 0,
 5,
 0,
 6,
 0,
 6,
 2,
 0,
 4,
 3,
 3,
 5,
 0,
 6,
 0,
 5,
 5,
 3,
 0,
 0,
 1,
 4,
 0,
 2,
 5,
 2,
 0,
 0,
 3,
 6,
 0,
 2,
 0,
 3,
 0,
 6,
 0,
 5,
 5,
 0,
 3,
 5,
 6,
 1,
 5,
 6,
 3,
 0,
 0,
 3,
 6,
 6,
 6,
 3,
 6,
 6,
 0,
 6,
 0,
 6,
 2,
 3,
 1,
 2,
 0,
 0,
 3,
 0,
 0,
 0,
 6,
 0,
 3,
 2,
 0,
 3,
 0,
 3,
 6,
 6,
 0,
 4,
 3,
 6,
 5,
 0,
 5,
 0,
 2,
 5,
 3,
 6,
 3,
 0,
 4,
 0,
 6,
 4,
 3,
 3,
 0,
 2,
 3,
 3,
 6,
 0,
 4,
 5,
 6,
 6,
 3,
 0,
 3,
 0,
 0,
 0,
 4,
 3,
 4,
 4,
 6,
 0,
 0,
 5,
 4,
 4,
 3,
 5,
 3,
 3,
 0,
 2,
 0,
 3,
 3,
 0,
 0,
 0,
 0,
 0,
 4,
 3,
 0,
 3,
 0,
 0,
 3,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 6,
 0,
 4,
 3,
 2,
 0,
 0,
 3,
 2,
 0,
 6,
 4,
 3,
 3,
 6,
 3,
 6,
 3,
 0,
 6,
 2,
 3,
 4,
 0,
 1,
 4,
 0,
 4,
 6,
 6,
 2,
 0,
 0,
 2,
 0,
 5,
 3,
 3,
 6,
 0,
 3,
 3,
 0,
 0,
 6,
 0,
 0,
 3,
 3,
 4,
 4,
 6,
 3,
 0,
 0,
 0,
 5,
 6,
 4,
 4,
 6,
 3,
 6,
 0,
 3,
 0,
 3,
 3,
 0,
 0,
 4,
 3,
 4,
 3,
 0,
 6,
 0,


In [34]:
y_labels = [np.argmax(x) for x in y_train]

In [35]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_labels, predicted_labels)

array([[1498,    0,   19,   63,   20,    4,   26],
       [  38,  137,   12,   44,   47,    2,   17],
       [  94,    0,  287,   96,   20,    8,   36],
       [ 242,    0,   63, 1329,  107,   26,   68],
       [ 104,    1,   17,  148,  876,    6,   23],
       [ 100,    0,   16,   59,   23,  420,   31],
       [ 144,    1,   16,   69,   20,    6, 1067]])

## Keras text classification - Request

In [16]:
train_size = int(len(df_requests) * .8)
train_posts = df_requests['Request'][:train_size]
train_tags = df_requests['Updated LOB'][:train_size]
test_posts = df_requests['Request'][train_size:]
test_tags = df_requests['Updated LOB'][train_size:]

vocab_size = 1000
tokenize = text.Tokenizer(num_words=vocab_size)
tokenize.fit_on_texts(train_posts)

x_train = tokenize.texts_to_matrix(train_posts)

encoder = LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

model = Sequential()

model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))

model.add(Dense(y_train.shape[1]))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

history = model.fit(x_train, y_train, 
                    batch_size=100, 
                    epochs=2, 
                    verbose=1, 
                    validation_split=0.1)

Train on 6705 samples, validate on 745 samples
Epoch 1/2
6705/6705 [==============================] - 9s 1ms/step - loss: 1.5965 - acc: 0.4710 - val_loss: 1.4002 - val_acc: 0.5315
Epoch 2/2
6705/6705 [==============================] - 8s 1ms/step - loss: 1.0624 - acc: 0.6513 - val_loss: 1.1424 - val_acc: 0.5893


## N Grams - LOB

In [27]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /Users/ZGS/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:


tf = TfidfVectorizer(smooth_idf=False, sublinear_tf=False, norm=None, analyzer='word')


txt_fitted = tf.fit(df_requests['Request Description'].values.astype('U'))
txt_transformed = txt_fitted.transform(df_requests['Request Description'].values.astype('U'))

train_tags = df_requests['Updated LOB']

encoder = LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)

print(txt_transformed.shape)
print(y_train.shape)




(9313, 14307)
(9313, 7)


In [16]:
model = Sequential()

model.add(Dense(512, input_shape=(14307,)))
model.add(Activation('relu'))

model.add(Dense(7))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

history = model.fit(txt_transformed, y_train, 
                    batch_size=100, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1)

Train on 8381 samples, validate on 932 samples
Epoch 1/10
8381/8381 [==============================] - 141s 17ms/step - loss: 1.2210 - acc: 0.5884 - val_loss: 1.1254 - val_acc: 0.6073
Epoch 2/10
8381/8381 [==============================] - 140s 17ms/step - loss: 0.2931 - acc: 0.9179 - val_loss: 1.2203 - val_acc: 0.6202
Epoch 3/10
8381/8381 [==============================] - 145s 17ms/step - loss: 0.1219 - acc: 0.9723 - val_loss: 1.3673 - val_acc: 0.6084
Epoch 4/10
8381/8381 [==============================] - 143s 17ms/step - loss: 0.0623 - acc: 0.9888 - val_loss: 1.5522 - val_acc: 0.6009
Epoch 5/10
8381/8381 [==============================] - 139s 17ms/step - loss: 0.0376 - acc: 0.9938 - val_loss: 1.6775 - val_acc: 0.6009
Epoch 6/10
8381/8381 [==============================] - 159s 19ms/step - loss: 0.0266 - acc: 0.9963 - val_loss: 1.7943 - val_acc: 0.5966
Epoch 7/10
8381/8381 [==============================] - 140s 17ms/step - loss: 0.0205 - acc: 0.9974 - val_loss: 1.8980 - val_acc: 0

In [28]:
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer = CountVectorizer(stop_words=sw,ngram_range=(1,2))
# train_posts = vectorizer.fit_transform(df_requests['Request Description'])
# train_posts.shape

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(stop_words='english',ngram_range=(1,4))
train_posts = vectorizer.fit_transform(df_requests['Request Description'].values.astype('U'))
train_posts.shape
#x = v.fit_transform(df['Review'].values.astype('U'))

(9313, 553007)

In [30]:
#
train_size = int(len(df_requests) * .8)
# train_posts = df_request['Request Description']#[:train_size]
# train_tags = df_requests['Updated LOB']#[:train_size]
# test_posts = df_requests['Request Description']#[train_size:]
# test_tags = df_requests['Updated LOB']#[train_size:]
# train_tags.shape

In [31]:
train_size = int(len(df_requests) * .8)
train_posts = df_requests['Request Description']#[:train_size]
train_tags = df_requests['Updated LOB']#[:train_size]
test_posts = df_requests['Request Description']#[train_size:]
test_tags = df_requests['Updated LOB']#[train_size:]
train_tags.shape

(9313,)

In [32]:
vocab_size = 8000
tokenize = text.Tokenizer(num_words=vocab_size)
tokenize.fit_on_texts(train_posts)

x_train = tokenize.texts_to_matrix(train_posts)
#print(x_train.shape)

print(x_train[0])
encoder = LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)
#y_train.shape

model = Sequential()
model.add(Dense(256, input_shape=(x_train.shape[1],)))
#model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(y_train.shape[1]))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

history = model.fit(x_train, y_train, 
                    batch_size=100, 
                    epochs=3, 
                    verbose=1, 
                    validation_split=0.1)

[0. 0. 1. ... 0. 0. 0.]
Train on 8381 samples, validate on 932 samples
Epoch 1/3
8381/8381 [==============================] - 39s 5ms/step - loss: 1.3961 - acc: 0.5339 - val_loss: 1.0915 - val_acc: 0.6599
Epoch 2/3
8381/8381 [==============================] - 41s 5ms/step - loss: 0.7111 - acc: 0.7839 - val_loss: 1.0183 - val_acc: 0.6352
Epoch 3/3
8381/8381 [==============================] - 39s 5ms/step - loss: 0.4508 - acc: 0.8690 - val_loss: 1.0496 - val_acc: 0.6330


## Export to Excel

In [17]:
df_requests.head()

,ID,Submission Date,Resolution Date,FRC Owner,Request,Requestor,Request Description,Status,Urgent?,Reopen?,...,Created,Created By,Modified,Last Modified By,Submission Month,res date,Comments/Notes,Item Type,Path,Updated LOB
0,14632,2018-10-18,2018-10-18,"Iannacone, Kristine",Account locked in TAP,"Melaniphy,Kevin M","\n​\nTeam,\n \nMy policy is locked in TAP, can...",Closed,No,No,...,2018-10-18 16:46:51,"Iannacone,Kristine",2018-10-18 17:02:04,"Iannacone,Kristine",201810,10/18/2018 17:02,"\nGood afternoon Kevin,\nThe TAP helpdesk will...",Item,ccm/FRC/Lists/TEST,Not LOB Specific
1,14631,2018-10-18,NaT,"Osswald, Ernst",Can we add Tow & Labor to auto without PPTs?,"Loquiao,Jeffrey E","\n​\nHello,\n \nCan you please confirm if we a...",Open,No,No,...,2018-10-18 16:42:54,"Iannacone,Kristine",2018-10-18 16:42:54,"Iannacone,Kristine",201810,-,NaN,Item,ccm/FRC/Lists/TEST,Auto
2,14630,2018-10-18,NaT,"Ewing, Jamie",BPP rates higher than Property rates,"Williams,Gina","\n​\nHello,\n \nCan you look into why the BPP ...",Open,No,No,...,2018-10-18 16:32:44,"Iannacone,Kristine",2018-10-18 16:32:44,"Iannacone,Kristine",201810,-,NaN,Item,ccm/FRC/Lists/TEST,Package Property
3,14629,2018-10-18,NaT,"Ewing, Jamie",Problem saving files to EPC/DMF,"Albright,Kevin T.",\n​I thought I read something recently on this...,Open,No,No,...,2018-10-18 16:25:37,"Iannacone,Kristine",2018-10-18 16:25:37,"Iannacone,Kristine",201810,-,NaN,Item,ccm/FRC/Lists/TEST,Package Property
4,14628,2018-10-18,NaT,"Iannacone, Kristine",Need point of contact for WC claims issues,"Sterling,Heather","\n​\nHello,\n \nI have a insured who would lik...",Open,No,No,...,2018-10-18 16:16:11,"Iannacone,Kristine",2018-10-18 16:16:11,"Iannacone,Kristine",201810,-,NaN,Item,ccm/FRC/Lists/TEST,Workers Comp


In [37]:
writer = pd.ExcelWriter('categories_output.xlsx')
df_list.to_excel(writer,'Sheet1')
writer.save()

In [18]:
fast_text = []
for row in df_requests['Updated LOB']:
    if row == 'Package General Liability':
        #fast_text.append('__label__Package-General-Liability')
        fast_text.append('__label__Package-General-Liability')
    elif row == 'Auto':
        fast_text.append('__label__Auto')
    elif row == 'Workers Comp':
        fast_text.append('__label__Workers-Comp')
    elif row =='Package Property':
        fast_text.append('__label__Package-Property')
    elif row == 'Umbrella':
        fast_text.append('__label__Umbrella')
    elif row == 'Not LOB Specific':
        fast_text.append('__label__Not-LOB-Specific')
    else:
        fast_text.append('__label__Monoline')

In [20]:
df_requests['Fast Text'] = fast_text
df_combined_text = df_requests[['Fast Text', 'Request Description']].apply(lambda x: ' '.join(x), axis=1)

In [22]:
type(df_combined_text)

pandas.core.series.Series